In [1]:
from sklearn.datasets import load_iris
# iris 데이터셋 로딩
X, y = load_iris(return_X_y=True)
print(X[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [2]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, stratify=y, shuffle=True)
print(len(X_train))

120


In [3]:
from torch import nn
#신경망 모형
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        #input layer, Linear 선형함수(1차함수)
        self.layer1 = nn.Linear(input_dim,50) #input nodes, output nodes 50
        self.layer2 = nn.Linear(50, 20)
        self.layer3 = nn.Linear(20, 3)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        # 출력층의 활성화함수 - 소프트맥스
        x = F.softmax(self.layer3(x), dim=0)
        return x

In [4]:
import torch
model = Model(X_train.shape[1]) # 초기화함수의 input_dim으로 변수개수가 전달됨
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # 최적화함수 정의
loss_fn = nn.CrossEntropyLoss() #손실함수 정의
epochs = 100

In [5]:
from torch.autograd import Variable
import torch.nn.functional as F
#넘파이배열로부터 텐서를 만들고
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
for epoch in range(1, epochs+1):
    print("Epoch",epoch)
    #예측값
    y_pred = model(X_train)
    #손실함수에 예측값과 실제값을 입력
    loss = loss_fn(y_pred, y_train)
    print('loss:',loss.item())

    # 경사 초기화
    optimizer.zero_grad()
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트

Epoch 1
loss: 1.098471760749817
Epoch 2
loss: 1.0975475311279297
Epoch 3
loss: 1.0966699123382568
Epoch 4
loss: 1.0957279205322266
Epoch 5
loss: 1.094659686088562
Epoch 6
loss: 1.0935672521591187
Epoch 7
loss: 1.0925476551055908
Epoch 8
loss: 1.091606855392456
Epoch 9
loss: 1.0908054113388062
Epoch 10
loss: 1.0901497602462769
Epoch 11
loss: 1.0896127223968506
Epoch 12
loss: 1.0892179012298584
Epoch 13
loss: 1.088807463645935
Epoch 14
loss: 1.0884815454483032
Epoch 15
loss: 1.0882140398025513
Epoch 16
loss: 1.0880019664764404
Epoch 17
loss: 1.0878649950027466
Epoch 18
loss: 1.0878080129623413
Epoch 19
loss: 1.0877872705459595
Epoch 20
loss: 1.0877412557601929
Epoch 21
loss: 1.0877189636230469
Epoch 22
loss: 1.0876667499542236
Epoch 23
loss: 1.0875988006591797
Epoch 24
loss: 1.087595820426941
Epoch 25
loss: 1.0876047611236572
Epoch 26
loss: 1.087586522102356
Epoch 27
loss: 1.087533712387085
Epoch 28
loss: 1.0874607563018799
Epoch 29
loss: 1.087393045425415
Epoch 30
loss: 1.08733224868774

In [6]:
# Prediction
X_test = Variable(torch.from_numpy(X_test)).float()
pred = model(X_test)
pred[:5]

tensor([[5.7648e-06, 1.7325e-02, 1.9829e-04],
        [4.0290e-02, 5.6586e-06, 4.6682e-09],
        [9.9983e-07, 1.4863e-03, 1.0729e-03],
        [2.0757e-07, 6.7924e-04, 3.2783e-03],
        [1.9193e-04, 1.3755e-01, 6.8063e-06]], grad_fn=<SliceBackward0>)

In [7]:
import numpy as np
np.argmax(pred.data.numpy(), axis=1)

array([1, 0, 1, 2, 1, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       2, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [8]:
from sklearn.metrics import accuracy_score
# 모형의 정확도 측정
accuracy_score(y_test, np.argmax(pred.data.numpy(), axis=1))

1.0

In [9]:
torch.save(model, "c:/data/iris/iris-torch.model")

In [10]:
model2 = torch.load("c:/data/iris/iris-torch.model")
np.argmax(model2(X_test[0]).data.numpy())

2

In [11]:
#pip install torchinfo
from torchinfo import summary
summary(model, input_size=(32, 4))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [32, 3]                   --
├─Linear: 1-1                            [32, 50]                  250
├─Linear: 1-2                            [32, 20]                  1,020
├─Linear: 1-3                            [32, 3]                   63
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
Total mult-adds (M): 0.04
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.02